In [12]:
import torch
import copy
#from geom3d.train_SchNet import SchNet, Pymodel, read_config, load_data, train_val_test_split
import numpy as np

config_dir = "/rds/general/user/cb1319/home/GEOM3D/Geom3D/training/SchNet_target_1K_TEST_5e4lr/"
chkpt_path = "/rds/general/user/cb1319/home/GEOM3D/Geom3D/training/SchNet_target_80K_TEST_5e4lr/epoch=91-val_loss=0.56-other_metric=0.00.ckpt"
config_dir = "/rds/general/user/cb1319/home/GEOM3D/Geom3D/training/SchNet_target_1K_TEST_5e4lr/"
chkpt_path = "/rds/general/user/cb1319/home/GEOM3D/Geom3D/training/SchNet_target_1K_TEST_5e4lr/epoch=2-val_loss=5.76-other_metric=0.00.ckpt"
#chkpt_path = "/rds/general/user/cb1319/home/GEOM3D/Geom3D/training/SchNet_target_1K_TEST_5e4lr/epoch=2-val_loss=1.17-other_metric=0.00.ckpt"

config = read_config(config_dir)
np.random.seed(config["seed"])
torch.cuda.manual_seed_all(config["seed"])
torch.manual_seed(config["seed"])

config["device"] = "cuda:0" if torch.cuda.is_available() else "cpu"

print('checkpoint used:', chkpt_path)



pymodel = Pymodel.load_from_checkpoint(chkpt_path, map_location={"cuda:0": "cpu"})
pymodel.freeze()

config loaded from /rds/general/user/cb1319/home/GEOM3D/Geom3D/training/SchNet_target_1K_TEST_5e4lr/
checkpoint used: /rds/general/user/cb1319/home/GEOM3D/Geom3D/training/SchNet_target_1K_TEST_5e4lr/epoch=2-val_loss=5.76-other_metric=0.00.ckpt


TypeError: Pymodel.__init__() missing 2 required positional arguments: 'model' and 'graph_pred_linear'

In [11]:
import pytorch_lightning as pl
class Pymodel(pl.LightningModule):
    def __init__(self, model, graph_pred_linear):
        super().__init__()
        self.save_hyperparameters(ignore=['graph_pred_linear', 'model'])
        self.molecule_3D_repr = model
        self.graph_pred_linear = graph_pred_linear

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        loss = self._get_preds_loss_accuracy(batch)

        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        """used for logging metrics"""
        loss = self._get_preds_loss_accuracy(batch)

        # Log loss and metric
        self.log("val_loss", loss)
        return loss

    def _get_preds_loss_accuracy(self, batch):
        """convenience function since train/valid/test steps are similar"""
        z = self.molecule_3D_repr(batch.x, batch.positions, batch.batch)
        z = self.graph_pred_linear(z)
        loss = Functional.mse_loss(z, batch.y.unsqueeze(1))
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=5e-4)
        return optimizer
    
    def forward(self, batch):
        z = self.molecule_3D_repr(batch.x, batch.positions, batch.batch)
        z = self.graph_pred_linear(z)
        return z

In [13]:
pymodel = Pymodel.load_from_checkpoint(chkpt_path)
pymodel.freeze()

RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
